In [0]:
### `alpha` Agent

This code defines a Databricks LangGraph agent for a life‑sciences knowledge base that:
Classifies the user’s question (with an “orchestrator” LLM node) into:
"sql" → question is about counts/aggregations/statistics.
"vector" → question needs semantic lookup in the knowledge base.
If routed to sql:
Uses Unity Catalog SQL functions (exposed as tools via UCFunctionToolkit) to:
Count rows in genes/proteins/pathways/compounds knowledge tables.
Count high‑confidence rows (given a min_confidence).
Compute average confidence.
Executes the tools via ToolNode, then calls the LLM again to produce a final natural language answer and returns it directly (no synthesizer step).
If routed to vector:
Runs two specialized vector workers in parallel using Send:
Vector Worker 1 → searches genes + proteins vector indexes.
Vector Worker 2 → searches pathways + compounds vector indexes.
Each worker:
Uses VectorSearchRetrieverTool tools bound to the LLM to issue vector‑search tool calls.
Executes those calls via ToolNode.
Collects raw search results.
Calls the LLM again to summarize the results (with emphasis on key findings and confidence).
Stores its own output plus metadata in state["worker_results"] (e.g. result text, confidence, source, number of tool calls).
Emits a status message like [Vector Worker X completed with N results].
Synthesizer/judge node:
Runs after both vector workers finish.
Reads all worker_results, formats them into a markdown‑style summary, and builds a “meta‑prompt”:
Explain the original question.
Show each worker’s result and confidence.
Asks the LLM to:
Evaluate relevance & confidence.
Reconcile any conflicts (favoring higher confidence).
Produce a single coherent, scientific final answer, citing which worker results it used.
Returns that synthesized answer as the final response.
Integration / serving:
Wraps the compiled LangGraph workflow into a custom LangGraphResponsesAgent that:
Implements predict and predict_stream in MLflow’s ResponsesAgent interface.
Converts inputs to OpenAI‑style chat format.
Streams intermediate node messages and final deltas back to the caller.
Registers this agent as the MLflow model (mlflow.models.set_model(AGENT)), with mlflow.langchain.autolog() enabled.
In short:
# It’s a multi‑worker, orchestrated agent that routes between SQL aggregations and parallel vector‑search workers over four life‑science knowledge bases, then synthesizes the vector workers’ findings into a single, high‑level scientific answer.

In [0]:
### `beta` Agent

# In summary: this agent is a multi-worker, UC- and vector-search-enabled orchestration graph, wrapped as an MLflow ResponsesAgent model. Architecturally it’s realistic (with routing, SQL tools, parallel retrievers, and a synthesizer), but the synthesizer is deliberately implemented to return a low-quality, generic answer.


This agent is a Databricks-hosted, LangGraph-based orchestration agent that routes user questions to different “workers” (sub-agents) and returns an answer through the MLflow ResponsesAgent interface. At a high level it does the following:
Takes chat-style input
It expects a list of messages (user/assistant) as input, in OpenAI-style format, via MLflow’s ResponsesAgentRequest.
It is wrapped as an MLflow pyfunc model using mlflow.models.set_model(AGENT), so you call it with mlflow.models.predict(...).
Orchestrates between two main paths: SQL vs. Vector search
The orchestrator_node looks at the last user message and decides:
"sql" for questions about counts, averages, or aggregate statistics over the life sciences tables.
"vector" for everything else that needs semantic search.
The orchestrator returns a routing decision plus a small debug message like "[Orchestrator: Routing to sql worker]".
Has a SQL worker that calls Unity Catalog functions
The SQL worker (sql_worker_node) handles analytic questions such as:
How many genes/proteins/pathways/compounds?
How many “high confidence” entries above a threshold?
What are average confidence scores?
It uses:
A UCFunctionToolkit with a set of Unity Catalog SQL functions (count_, count_high_confidence_, avg_confidence_*).
A Databricks LLM (ChatDatabricks) bound to these tools, so the LLM decides which UC function(s) to call.
Execution flow:
LLM is prompted with the list of UC functions and the user’s question.
If the LLM emits tool calls, a ToolNode executes those UC functions.
The LLM is then invoked again with the original context plus the tool outputs to generate a natural-language answer.
The node returns a final AIMessage containing that answer.
Has two parallel vector search workers (multi-worker retrievers)
If the orchestration route is "vector", it sends the state to:
vector_worker_1_node (genes + proteins),
vector_worker_2_node (pathways + compounds), in parallel using Send(...) from LangGraph.
Each worker:
Uses VectorSearchRetrieverTool configured to search the respective Unity Catalog vector search indexes for its domain.
Prompts the LLM with a “you are Vector Worker X” instruction and the user question.
Lets the LLM call the retriever tools to fetch semantically relevant entries.
Optionally asks the LLM again to summarize the retrieved items.
Stores a structured result in worker_results (with a “confidence” and a human-readable summary) and emits a debug message like "[Vector Worker 1 completed with N results]".
Uses a synthesizer node – but here intentionally “bad”
For vector questions, both vector workers feed into a synthesizer_node.
In a normal design, the synthesizer would:
Read all worker_results,
Combine them,
And produce a coherent, source-aware, high-quality answer.
In this “BAD” version:
The synthesizer ignores the worker results and the user’s question.
It always returns a fixed, unhelpful answer:
"I don't know. I cannot provide a detailed answer to this question right now."
Structurally, it’s correct (the graph and messaging shape stay compatible), but behaviorally it’s intentionally poor.
Graph structure (LangGraph)
Defined over AgentState with:
messages (conversation history),
worker_results (aggregated results from vector workers),
route_decision (sql vs. vector).
Nodes:
orchestrator (entry point),
sql_worker,
vector_worker_1,
vector_worker_2,
synthesizer.
Edges:
orchestrator → conditionally:
sql_worker then END, or
both vector_worker_1 and vector_worker_2 in parallel.
Both vector workers → synthesizer → END.
MLflow ResponsesAgent wrapper
LangGraphResponsesAgent adapts the LangGraph app to the MLflow ResponsesAgent protocol:
predict_stream:
Runs agent.stream(..., stream_mode=["updates", "messages"]).
For each node that emits messages, converts them into ResponsesAgentStreamEvent items via output_to_responses_items_stream.
Also streams AIMessageChunk deltas as text deltas.
predict:
Consumes the stream, collects items where event.type == "response.output_item.done".
Returns a ResponsesAgentResponse with output=[...] and any custom_outputs.
This means the model returns a list of assistant messages (including debug/trace messages from orchestrator and workers), each with content chunks of type output_text.

In [0]:
This code defines a Databricks LangGraph-based “orchestrator” agent for a life‑sciences knowledge base, then intentionally makes it behave badly at the final step.
Below is what each major part does and how the whole thing behaves end‑to‑end.
High‑level behavior
Takes a user question (via ResponsesAgent / MLflow model).
Orchestrator LLM decides a route:
"sql" → use SQL/UC functions for counts and aggregates.
"vector" → use vector search (semantic search) over genes/proteins/pathways/compounds.
If sql:
A SQL worker uses Databricks UC functions (UCFunctionToolkit) as tools to answer simple statistics questions and returns a direct answer.
If vector:
Two vector workers run (in parallel in graph terms):
Vector Worker 1 → genes + proteins vector search tools.
Vector Worker 2 → pathways + compounds vector search tools.
Each worker:
Calls the appropriate vector search tools.
Gathers raw results.
Asks the LLM to summarize them.
Stores its own summary + metadata in worker_results.
Then a synthesizer node is called to combine / judge worker outputs, but in this implementation it ignores everything and always returns the same bad answer:
"I don't know. I cannot provide a detailed answer to this question right now."
So functionally:
SQL questions → get a normal, data‑driven answer from the SQL worker.
Semantic/vector questions → the two vector workers do all the right work, but the final answer the user sees is always that canned “I don’t know” message. The vector work is effectively thrown away.
The structure is correct, but the agent is deliberately “broken” at the synthesizer step.
Configuration
ExamplePython
LLM_ENDPOINT_NAME = "databricks-claude-3-7-sonnet"
CATALOG = "mmt"
SCHEMA = "LS_agent"
VECTOR_SEARCH_ENDPOINT = "ls_vs_mmt"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)
Uses Databricks’ ChatDatabricks LLM endpoint (databricks-claude-3-7-sonnet).
All UC functions and vector indexes are under mmt.LS_agent.*.
Tools
SQL / UC function tools
Built from UCFunctionToolkit:
ExamplePython
UC_TOOL_NAMES = [
    "mmt.LS_agent.count_genes_knowledge_rows",
    "mmt.LS_agent.count_proteins_knowledge_rows",
    ...
    "mmt.LS_agent.avg_confidence_compounds_knowledge",
]
uc_toolkit = UCFunctionToolkit(function_names=UC_TOOL_NAMES)
sql_tools = uc_toolkit.tools
These are predefined UC scalar functions for:
Counting rows per table (genes, proteins, pathways, compounds).
Counting high‑confidence rows (param: min_confidence).
Getting average confidence.
Used only by the SQL worker.
Vector search tools
ExamplePython
VECTOR_SEARCH_TOOLS = [
    VectorSearchRetrieverTool(... name="search_genes" ...),
    VectorSearchRetrieverTool(... name="search_proteins" ...),
    VectorSearchRetrieverTool(... name="search_pathways" ...),
    VectorSearchRetrieverTool(... name="search_compounds" ...),
]
Each tool:
Uses a Unity Catalog vector search index, e.g.:
mmt.LS_agent.genes_knowledge_vs_index
mmt.LS_agent.proteins_knowledge_vs_index
Returns top‑k (3) results with fields ["id", "name", "description", "confidence"].
Used by Vector Worker 1 (genes, proteins) and Vector Worker 2 (pathways, compounds).
Agent state
ExamplePython
class AgentState(TypedDict):
    messages: Annotated[Sequence[AnyMessage], add_messages]
    custom_inputs: Optional[dict[str, Any]]
    custom_outputs: Optional[dict[str, Any]]
    worker_results: Annotated[Optional[dict[str, Any]], merge_worker_results]
    route_decision: Optional[str]
messages: chat history, merged with add_messages (LangGraph pattern).
worker_results: dictionary merged via merge_worker_results so each worker can add its own result.
route_decision: set by the orchestrator to "sql" or "vector".
Orchestrator node
ExamplePython
def orchestrator_node(state: AgentState, config: RunnableConfig):
    ...
    orchestrator_prompt = """You are an orchestrator for a life sciences knowledge base system.
    ...
    Respond with ONLY one word: either "sql" or "vector"
    """
    response = llm.invoke(decision_messages)
    decision = response.content.strip().lower()
    ...
    return {
        "route_decision": decision,
        "messages": [AIMessage(content=f"[Orchestrator: Routing to {decision} worker]")]
    }
Looks only at the last user message.
Uses LLM to classify:
Count/avg/statistics questions → "sql".
Everything else → "vector".
Writes a system‑style message indicating which worker is chosen.
SQL worker node
ExamplePython
def sql_worker_node(state: AgentState, config: RunnableConfig):
    ...
    sql_llm = llm.bind_tools(sql_tools)
    response = sql_llm.invoke(sql_messages)
    ...
    if response.tool_calls:
        tool_node = ToolNode(sql_tools)
        tool_results = tool_node.invoke({"messages": [response]})
        ...
        final_response = llm.invoke(sql_messages + result_messages)
    ...
    return {"messages": [AIMessage(content=final_answer)]}
Flow:
System prompt describes available UC functions and their role.
LLM with sql_tools bound:
Can produce tool calls like count_genes_knowledge_rows().
ToolNode(sql_tools) actually executes those functions.
LLM called again with:
Original prompt + user + tool results
To compose a natural language answer.
Returns a single AIMessage with that answer.
For SQL route, this is the final answer (graph edge goes to END).
Vector worker 1: genes & proteins
ExamplePython
def vector_worker_1_node(state: AgentState, config: RunnableConfig):
    worker_tools = [VECTOR_SEARCH_TOOLS[0], VECTOR_SEARCH_TOOLS[1]]
    worker_llm = llm.bind_tools(worker_tools)
    response = worker_llm.invoke(worker_messages)
    ...
    if response.tool_calls:
        tool_node = ToolNode(worker_tools)
        tool_results = tool_node.invoke({"messages": [response]})
        ...
        final_response = llm.invoke(
            worker_messages
            + result_messages
            + [SystemMessage(content="Summarize the search results...")]
        )
    ...
    worker_results["vector_worker_1"] = {
        "result": final_result,
        "confidence": 0.80,
        "source": "Vector Worker 1 (Genes & Proteins)",
        "tool_calls_made": len(response.tool_calls) if response.tool_calls else 0,
    }
    return {
        "worker_results": worker_results,
        "messages": [AIMessage(content=f"[Vector Worker 1 completed with {len(search_results)} results]")],
    }
Uses search_genes and search_proteins.
Executes those tools via ToolNode.
Extracts and concatenates raw search results.
Calls LLM once more to create a summary of search results.
Stores its final text + metadata in state["worker_results"]["vector_worker_1"].
Emits a status message like:
[Vector Worker 1 completed with N results]
Vector worker 2: pathways & compounds
Same pattern, but with search_pathways and search_compounds and stores under "vector_worker_2".
Synthesizer node (intentionally bad)
ExamplePython
def synthesizer_node(state: AgentState, config: RunnableConfig):
    """
    BAD synthesizer: ignores worker_results and always answers poorly.
    """
    bad_answer = (
        "I don't know. I cannot provide a detailed answer to this question right now."
    )
    return {
        "messages": [AIMessage(content=bad_answer)]
    }
Ignores:
state["worker_results"]
Vector worker summaries
Any intermediate messages
Always returns the same generic “I don’t know” answer.
This is why the comment calls the agent “BAD behavior here”.
In a proper implementation, this node would:
Read state["worker_results"].
Compare confidence, tools used, content.
Synthesize a final, helpful answer from the vector workers’ outputs.
Routing logic and graph structure
ExamplePython
def route_after_orchestrator(state: AgentState):
    decision = state.get("route_decision", "vector")
    if decision == "sql":
        return Send("sql_worker", state)
    else:
        return [
            Send("vector_worker_1", state),
            Send("vector_worker_2", state),
        ]
Graph wiring:
ExamplePython
workflow.add_node("orchestrator", orchestrator_node)
workflow.add_node("sql_worker", sql_worker_node)
workflow.add_node("vector_worker_1", vector_worker_1_node)
workflow.add_node("vector_worker_2", vector_worker_2_node)
workflow.add_node("synthesizer", synthesizer_node)

workflow.set_entry_point("orchestrator")
workflow.add_conditional_edges("orchestrator", route_after_orchestrator)
workflow.add_edge("sql_worker", END)
workflow.add_edge("vector_worker_1", "synthesizer")
workflow.add_edge("vector_worker_2", "synthesizer")
workflow.add_edge("synthesizer", END)
orchestrator → either:
sql_worker → END
or
vector_worker_1 & vector_worker_2 → synthesizer → END.
Workers share state through worker_results (using merge_worker_results).
MLflow / ResponsesAgent wrapper
ExamplePython
class LangGraphResponsesAgent(ResponsesAgent):
    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        ...
    def predict_stream(self, request: ResponsesAgentRequest) -> Generator[ResponsesAgentStreamEvent, None, None]:
        ...
Wraps the LangGraph agent into MLflow’s ResponsesAgent interface.
predict_stream:
Converts input to OpenAI‑style chat format with to_chat_completions_input.
Calls agent.stream(..., stream_mode=["updates", "messages"]).
Streams back messages and deltas via ResponsesAgentStreamEvent.
predict collects the final output items from the stream.
Finally:
ExamplePython
mlflow.langchain.autolog()
agent = create_orchestrator_agent()
AGENT = LangGraphResponsesAgent(agent)
mlflow.models.set_model(AGENT)
Enables LangChain autologging.
Creates the graph.
Wraps it.
Registers it as the MLflow model object.
Summary in one sentence
# This code implements a LangGraph‑based orchestrator–worker agent on Databricks that routes questions between SQL/UC tools and multiple vector‑search workers for a life‑sciences knowledge base, but intentionally uses a “broken” synthesizer that discards vector worker results and always replies with “I don’t know.”